In [1]:
import numpy as np
import matplotlib.pyplot as plt
from keras.layers import Dense,Conv2D,MaxPool2D,Dropout,Flatten
from keras.models import Sequential
from keras.preprocessing import image

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import shutil
import math
import imutils
import cv2

In [4]:
root_dir = r'coviddataset'

num_of_images = {}

for i in os.listdir(root_dir):
    num_of_images[i] = len(os.listdir(os.path.join(root_dir , i)))
        
num_of_images

{'COVID': 1252, 'non-COVID': 1229}

In [5]:
if not os.path.exists(root_dir+'\\train'):
    os.mkdir(root_dir+'\\train')
    
    for dirty in os.listdir(root_dir):
        if (dirty == 'train') | (dirty=='test'):
            pass
        else:
            os.mkdir(root_dir+'\\train/'+dirty)
            for img in np.random.choice(a = os.listdir(os.path.join(root_dir,dirty)),
                                       size = (math.floor(70/100*num_of_images[dirty])-5) , replace = False):
                src = os.path.join(root_dir,dirty, img)
                dest = os.path.join(root_dir+'\\train', dirty)
                shutil.copy(src , dest)
                os.remove(src)
else:
    print("Train Folder Exists")

In [6]:
if not os.path.exists(root_dir+'\\test'):
    os.mkdir(root_dir+'\\test')
    
    for dirty in os.listdir(root_dir):
        if (dirty == 'train') | (dirty=='test'):
            pass
        else:
            os.mkdir(root_dir+'\\test/'+dirty)
            for img in np.random.choice(a = os.listdir(os.path.join(root_dir,dirty)),
                                       size = (math.floor(15/100*num_of_images[dirty])-5) , replace = False):
                src = os.path.join(root_dir,dirty, img)
                dest = os.path.join(root_dir+'\\test', dirty)
                shutil.copy(src , dest)
                os.remove(src)
else:
    print("Test Folder Exists")

In [7]:
if not os.path.exists(root_dir+'\\val'):
    os.mkdir(root_dir+'\\val')
    
    for dirty in os.listdir(root_dir):
        if (dirty == 'train') | (dirty=='test') | (dirty == 'val'):
            pass
        else:
            os.mkdir(root_dir+'\\val/'+dirty)
            for img in np.random.choice(a = os.listdir(os.path.join(root_dir,dirty)),
                                       size = (math.floor(15/100*num_of_images[dirty])-5) , replace = False):
                src = os.path.join(root_dir,dirty, img)
                dest = os.path.join(root_dir+'\\val', dirty)
                shutil.copy(src , dest)
                os.remove(src)
else:
    print("Val Folder Exists")

In [27]:
train_datagen = image.ImageDataGenerator(
        rescale = 1./255,horizontal_flip = True,zoom_range = 0.2,shear_range = 0.2
)

train_data = train_datagen.flow_from_directory(directory = r'coviddataset\train',
                                              target_size = (256,256) , batch_size = 16 , class_mode = 'binary')

Found 1726 images belonging to 2 classes.


In [28]:
train_data.class_indices

{'COVID': 0, 'non-COVID': 1}

In [29]:
test_datagen = image.ImageDataGenerator(rescale = 1./255)

test_data = test_datagen.flow_from_directory(directory = r'coviddataset\test',
                                              target_size = (256,256) , batch_size = 16 , class_mode = 'binary')

Found 361 images belonging to 2 classes.


In [30]:
val_datagen = image.ImageDataGenerator(rescale = 1./255)

val_data = val_datagen.flow_from_directory(directory = r'coviddataset\val',
                                              target_size = (256,256) , batch_size = 16 , class_mode = 'binary')

Found 361 images belonging to 2 classes.


In [31]:
model = Sequential()
model.add(Conv2D(filters = 32, kernel_size = (3,3) , activation = 'relu' , input_shape = (256,256,3)))

model.add(Conv2D(filters = 64 , kernel_size = (3,3) , activation = 'relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64 , kernel_size = (3,3) , activation = 'relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))

model.add(Conv2D(filters = 128 , kernel_size = (3,3) , activation = 'relu'))
model.add(MaxPool2D())
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(units = 64, activation = 'relu'))
model.add(Dropout(0.50))
model.add(Dense(units = 1, activation = 'sigmoid'))

model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_8 (Conv2D)            (None, 254, 254, 32)      896       
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 252, 252, 64)      18496     
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
dropout_8 (Dropout)          (None, 126, 126, 64)      0         
_________________________________________________________________
conv2d_10 (Conv2D)           (None, 124, 124, 64)      36928     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 62, 62, 64)        0         
_________________________________________________________________
dropout_9 (Dropout)          (None, 62, 62, 64)       

In [17]:
import keras

In [36]:
model.compile(optimizer = 'adam' , loss = keras.losses.binary_crossentropy, metrics = ['accuracy'])

In [33]:
from keras.callbacks import ModelCheckpoint , EarlyStopping

In [34]:
es = EarlyStopping(monitor= 'val_accuracy', min_delta = 0.01, patience = 5,verbose = 1 , mode = 'auto')

mc = ModelCheckpoint(filepath = 'coviddetectcnn_best_model.h5' , monitor= 'val_accuracy', min_delta = 0.01, patience = 5,verbose = 1 ,
                     mode = 'auto', save_best_only = True)

callback = [es,mc]

In [35]:
hist = model.fit(train_data , steps_per_epoch = 20 ,epochs = 30 , verbose = 1, validation_data = val_data,
                          validation_steps = 16 , callbacks = callback) 

Epoch 1/30
20/20 [==============================] - 23s 988ms/step - loss: 1.2524 - accuracy: 0.4497 - val_loss: 0.6915 - val_accuracy: 0.5312

Epoch 00001: val_accuracy improved from -inf to 0.53125, saving model to coviddetectcnn_best_model.h5
Epoch 2/30
20/20 [==============================] - 18s 901ms/step - loss: 0.6897 - accuracy: 0.5344 - val_loss: 0.6930 - val_accuracy: 0.4883

Epoch 00002: val_accuracy did not improve from 0.53125
Epoch 3/30
20/20 [==============================] - 18s 886ms/step - loss: 0.6951 - accuracy: 0.5344 - val_loss: 0.6930 - val_accuracy: 0.5039

Epoch 00003: val_accuracy did not improve from 0.53125
Epoch 4/30
20/20 [==============================] - 18s 881ms/step - loss: 0.6833 - accuracy: 0.5531 - val_loss: 0.6791 - val_accuracy: 0.5078

Epoch 00004: val_accuracy did not improve from 0.53125
Epoch 5/30
20/20 [==============================] - 18s 900ms/step - loss: 0.7008 - accuracy: 0.5188 - val_loss: 0.6930 - val_accuracy: 0.4961

Epoch 00005: 

In [54]:
mc1 = ModelCheckpoint(filepath = 'coviddetectcnn_best_model.h5' , monitor= 'val_accuracy', min_delta = 0.01, patience = 5,verbose = 1 ,
                     mode = 'auto', save_best_only = True)

es1 = EarlyStopping(monitor= 'val_accuracy', min_delta = 0.01, patience = 5,verbose = 1 , mode = 'auto')

callback1 = [es1, mc1]

hist = model.fit(train_data , steps_per_epoch = 20 ,epochs = 30 , verbose = 1, validation_data = val_data,
                          validation_steps = 2 , callbacks = callback1) 

Epoch 1/30
20/20 [==============================] - 17s 787ms/step - loss: 0.4391 - accuracy: 0.7844 - val_loss: 0.3254 - val_accuracy: 0.8438

Epoch 00001: val_accuracy improved from -inf to 0.84375, saving model to coviddetectcnn_best_model.h5
Epoch 2/30
20/20 [==============================] - 15s 761ms/step - loss: 0.5510 - accuracy: 0.7312 - val_loss: 0.4918 - val_accuracy: 0.9062

Epoch 00002: val_accuracy improved from 0.84375 to 0.90625, saving model to coviddetectcnn_best_model.h5
Epoch 3/30
20/20 [==============================] - 16s 794ms/step - loss: 0.4813 - accuracy: 0.7358 - val_loss: 0.4094 - val_accuracy: 0.8750

Epoch 00003: val_accuracy did not improve from 0.90625
Epoch 4/30
20/20 [==============================] - 16s 778ms/step - loss: 0.4542 - accuracy: 0.7906 - val_loss: 0.3995 - val_accuracy: 0.7500

Epoch 00004: val_accuracy did not improve from 0.90625
Epoch 5/30
20/20 [==============================] - 15s 769ms/step - loss: 0.4676 - accuracy: 0.7812 - val_

In [55]:
hist.history.keys()

dict_keys(['loss', 'accuracy', 'val_loss', 'val_accuracy'])

In [56]:
hist.history['val_accuracy']

[0.84375,
 0.90625,
 0.875,
 0.75,
 0.9375,
 0.90625,
 0.875,
 0.78125,
 0.90625,
 0.78125]

In [57]:
from keras.models import load_model

In [58]:
model = load_model(r'coviddetectcnn_best_model.h5')

In [88]:
path = 'D:\PYTHON_OVERALL\jupyter files\coviddataset\COVID\Covid (1074).png'
img = image.load_img(path , target_size = (256,256))

img = image.img_to_array(img)/255.
img = np.array([img])
img.shape

(1, 256, 256, 3)

In [87]:
os.listdir(r'coviddataset\COVID')

['Covid (1074).png',
 'Covid (1207).png',
 'Covid (296).png',
 'Covid (299).png',
 'Covid (320).png',
 'Covid (372).png',
 'Covid (373).png',
 'Covid (375).png',
 'Covid (549).png',
 'Covid (55).png',
 'Covid (594).png',
 'Covid (638).png',
 'Covid (663).png',
 'Covid (669).png',
 'Covid (801).png',
 'Covid (869).png',
 'Covid (924).png']

In [89]:
pred = model.predict(img)

In [90]:
if pred<0.5:
    print("He has covid")
else:
    print("He is healthy")

He has covid


In [93]:
dirt = r'coviddataset\COVID'
covid = []
for i in os.listdir(dirt):
    path = os.path.join(dirt , i)
    img = image.load_img(path , target_size = (256,256))
    img = image.img_to_array(img)/255.
    img = np.array([img])
    pred = model.predict(img)
    if pred < 0.5:
        covid.append(0)
    else:
        covid.append(1)

In [94]:
covid

[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1]

In [95]:
dirt = r'coviddataset\non-COVID'
nocovid = []
for i in os.listdir(dirt):
    path = os.path.join(dirt , i)
    img = image.load_img(path , target_size = (256,256))
    img = image.img_to_array(img)/255.
    img = np.array([img])
    pred = model.predict(img)
    if pred < 0.5:
        nocovid.append(0)
    else:
        nocovid.append(1)